# Tâche #1 : Prédiction du score NOVA d'aliments avec des modèles *Transformers*

On reprend, comme au premier travail, la tâche de prédiction du score NOVA de produits alimentaires. Cependant, des nouveaux jeux de données ont été produits. Le corpus de textes contient 3 partitions : 
-	Un fichier d’entraînement -  *data/t1_nova_train.json*
-	Un fichier de validation -  *data/t1_nova_dev.json*
-	Un fichier de test - *data/t1_nova_test.json*
 
Utilisez la librairie *HuggingFace* pour accomplir cette tâche. On vous demande plus spécifiquement d’utiliser 2 modèles: le modèle ***bert-base-uncased*** et un **modèle encodeur multilingue** de votre choix. 

Les consignes pour cette tâche sont : 
- Nom du notebook : *t1_classification_nova.ipynb* (ce notebook). 
- Tokenisation : Celle fournie par les tokeniseurs accompagnant les modèles transformers. 
- Plongements de mots : Ceux du modèle *transformer*. 
- Normalisation : Lettre en minuscule pour Bert (rien à faire, le tokenizer s’en occupe). Aucune contrainte pour le 2e modèle mais il est préférable de comparer les modèles sur une même base (lettres minuscules). 
- Choix du 2e transformer: Un modèle encodeur multilingue préentraîné. Le modèle peut être une variante de Bert, mais cela n'est pas exigé. Me consulter en cas de doute pour valider votre choix.
- Entraînement : Un affinage (*fine-tuning*) des modèles encodeurs. Pas de pré-entraînement (*no further pretraining*) demandé pour cette tâche. 
- Analyse : Présentez clairement vos résultats et faites-en l’analyse. Comparez les résultats obtenus avec les 2 modèles.    

Vous pouvez ajouter au *notebook* toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2, etc.) si cela améliore la lisibilité. 

Vous pouvez diviser cette tâche en 2 *notebooks* (un pour chaque modèle) si cela est plus simple pour vous. Dans ce cas, ajoutez le nom du modèle dans le nom du fichier (par ex. *t1_classification_nova_bert.ipynb*) et retirez les cellules prévues pour le 2e modèle.  

Notes :
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si non trivial).
- Analyser vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc. 
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement des modèles.
- Vous pouvez ajouter des cellules au *notebook* comme bon vous semble.

Si nécessaire, installez les *packages* suivant. Si vous exécutez sur Code Colab, ces *packages* devraient déjà être installés.

In [1]:
#!pip install datasets
#!pip install accelerate
#!pip install "transformers[torch]"
#!pip3 install torch torchvision
#!pip install --upgrade transformers

## 1. Création du jeu de données (*les 3 partitions du dataset*)

In [40]:
train_json_fn = "./data/t1_nova_train.json"  # Jeu de données d'entraînement
dev_json_fn = "./data/t1_nova_dev.json"  # Jeu de données de validation
test_json_fn = "./data/t1_nova_test.json"  # Jeu de données de test

In [41]:
from datasets import Dataset

label2id = {"Groupe 1": 0, "Groupe 2": 1, "Groupe 3": 2, "Groupe 4": 3}
id2label = {0: "Groupe 1", 1: "Groupe 2", 2: "Groupe 3", 3: "Groupe 4"}

def load_data(json_fn):
    dataset = Dataset.from_json(json_fn)
    dataset = dataset.map(lambda nova_group: {"labels": label2id[nova_group["nova"]]})
    return dataset

In [42]:
train_dataset = load_data(train_json_fn)
dev_dataset = load_data(dev_json_fn)
test_dataset = load_data(test_json_fn)

## 2. Classification des scores NOVA avec BERT

### 2.1 Modèle BERT


In [43]:
model_id = "bert-base-uncased"

In [46]:
from transformers import AutoTokenizer

def tokenize_dataset(data_set, tokenizer):
    return data_set.map(lambda examples: tokenizer(examples['text'], padding="max_length", truncation=True))

def tokenize_all_dataset(train_dataset, dev_dataset, test_dataset):
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    train_dataset = tokenize_dataset(train_dataset, tokenizer)
    dev_dataset = tokenize_dataset(dev_dataset, tokenizer)
    test_dataset = tokenize_dataset(test_dataset, tokenizer)
    return train_dataset, dev_dataset, test_dataset

In [47]:
train_dataset, dev_dataset, test_dataset = tokenize_all_dataset(train_dataset, dev_dataset, test_dataset)

Map: 100%|██████████| 2126/2126 [00:00<00:00, 2761.74 examples/s]


### 2.2 Entraînement du modèle BERT


In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=4, id2label=id2label, label2id=label2id)

args = TrainingArguments(
    output_dir="mon_modele_nova",   # Dossier de sauvegarde
    eval_strategy="epoch",          # Évaluer à la fin de chaque époque
    save_strategy="epoch",          # Sauvegarder à la fin de chaque époque
    learning_rate=2e-5,             # Vitesse d'apprentissage (standard pour BERT)
    per_device_train_batch_size=8,  # Taille des lots (baisser si erreur mémoire GPU)
    per_device_eval_batch_size=8,
    num_train_epochs=3,             # Nombre de fois qu'il voit toutes les données
    weight_decay=0.01,              # Régularisation
    load_best_model_at_end=True,    # Garder le meilleur modèle à la fin, pas le dernier
    metric_for_best_model="accuracy"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dev_dataset,
    eval_dataset=test_dataset
)

### 2.3 Évaluation du modèle BERT

## 3. Classification des scores NOVA avec un encodeur multilingue

### 3.1  Deuxième modèle - Un encodeur multilingue 

Décrivez ici le modèle que vous avez choisi. (À compléter)

### 3.2 Entraînement du deuxième modèle


### 3.3 Évaluation du deuxième modèle

## 4. Analyse des résultats et comparaison des 2 modèles